## Encontrando Sinonimos e Analogias

Esse notebook tem o objetivo de codificar (encode) as palavras utilizando um modelo pretreinado. Cada palavra sera codificada em um vetor de dim = 50 (pré-estabelecido pelo modelo). Dessa forma, com as palavras em uma dimensão vetorizada podemos encontrar similaridade entre elas, ou seja, encontrar sinônimos e analogias.

Para ser mais objetivo, a palavra 'chip' por exemplo, sera codificada em um vetor de dim = 50 ou seja :

chip  =  [-0.7710, -1.1697,  1.5195,  0.8371,  0.7419, -0.2185, -0.7212, -0.9400,-0.0113,  0.5485,  0.4040, -0.1846, -0.4630,  0.2620, -0.6464,  0.3599,
-0.8610, -0.3869, -0.0271, -1.0254,  0.3280, -0.7500, -0.6859, -0.6912, 0.3429, -0.6660, -0.2910, -0.6104,  0.3322, -0.4252,  2.4573, -0.8748, 0.4891,  1.2888,  0.5780, -0.5509, -0.2263,  0.8127,  0.7048, -0.5498, 0.3620, -0.2171, -0.2991,  0.2917,  1.2260,  0.2446,  1.2133, -0.0967, 0.0474,  0.1971]


In [ ]:
!pip install -U d2l
from d2l import torch as d2l
import torch
from torch import nn
import os

     |████████████████████████████████| 81kB 4.1MB/s 


In [ ]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [ ]:
d2l.DATA_HUB['glove.6b.50d']

('http://d2l-data.s3-accelerate.amazonaws.com/glove.6B.50d.zip',
 '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

In [ ]:
class TokenEmbedding:
    def __init__(self, embedding_name):
      self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_name)
      self.unknown_idx = 0
      self.token_to_idx = {token: idx for idx, token in 
                         enumerate(self.idx_to_token)}
    
    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
      indices = [self.token_to_idx.get(token, self.unknown_idx) for token in tokens]
      vecs = self.idx_to_vec[torch.tensor(indices)]
      return vecs

    def __len__(self):
      return len(self.idx_to_token)

In [ ]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [ ]:
glove_6b50d.token_to_idx['science'], glove_6b50d.idx_to_token[1122]

(1122, 'science')

In [ ]:
glove_6b50d[['chip']]

tensor([[-0.7710, -1.1697,  1.5195,  0.8371,  0.7419, -0.2185, -0.7212, -0.9400,
         -0.0113,  0.5485,  0.4040, -0.1846, -0.4630,  0.2620, -0.6464,  0.3599,
         -0.8610, -0.3869, -0.0271, -1.0254,  0.3280, -0.7500, -0.6859, -0.6912,
          0.3429, -0.6660, -0.2910, -0.6104,  0.3322, -0.4252,  2.4573, -0.8748,
          0.4891,  1.2888,  0.5780, -0.5509, -0.2263,  0.8127,  0.7048, -0.5498,
          0.3620, -0.2171, -0.2991,  0.2917,  1.2260,  0.2446,  1.2133, -0.0967,
          0.0474,  0.1971]])

## Applying Pretrained Word Vectors

### Encontrando Sinonimos. 

Usaremos o methodo k-nearest neighbors para procurar sinônimos.

In [ ]:
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [ ]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

In [ ]:
topk, cos = knn(glove_6b50d.idx_to_vec, glove_6b50d[['chip']], 3 + 1)

In [ ]:
get_similar_tokens('chip', 4, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics
cosine sim=0.731: semiconductor


In [ ]:
get_similar_tokens('science', 4, glove_6b50d)

cosine sim=0.855: sciences
cosine sim=0.844: research
cosine sim=0.839: institute
cosine sim=0.837: studies


In [ ]:
get_similar_tokens('queen', 4, glove_6b50d)

cosine sim=0.852: princess
cosine sim=0.805: lady
cosine sim=0.787: elizabeth
cosine sim=0.784: king


### Cosseno calculo separado

In [ ]:
glove_6b50d[['chip']].reshape(-1,).shape, glove_6b50d.idx_to_vec.shape

(torch.Size([50]), torch.Size([400001, 50]))

In [ ]:
cos = torch.mv(glove_6b50d.idx_to_vec, glove_6b50d[['chip']].reshape(-1,)) / (
        torch.sqrt(torch.sum(glove_6b50d.idx_to_vec * glove_6b50d.idx_to_vec, axis=1) + 1e-9) *
        torch.sqrt((glove_6b50d[['chip']] * glove_6b50d[['chip']]).sum()))
cos

tensor([ 0.0000,  0.3935,  0.4617,  ..., -0.2265, -0.1674, -0.3025])

### Encontrando Analogia

In [ ]:
def get_analogy(token_a, token_b, token_c, embed):
  vecs = embed[[token_a, token_b, token_c]]
  x = vecs[1] - vecs[0] + vecs[2]
  topk, cos = knn(embed.idx_to_vec, x , 1)
  return embed.idx_to_token[int(topk[0])]

In [ ]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [ ]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'